# Cohere agent with LangChain

Install Libraries

In [6]:
# ! pip install streamlit
# ! pip install qdrant-client
# ! pip install langchain
# ! pip install langchain-community
# ! pip install langchain-qdrant
# ! pip install langchain-cohere
# ! pip install tiktoken
# ! pip install cohere
# ! pip install bs4

Ignore Warnings

In [7]:
import warnings
warnings.filterwarnings('ignore')

Import API-Keys

In [23]:
import streamlit as st
import os


ALPHAVANTAGE_API_KEY = st.secrets["ALPHAVANTAGE_API_KEY"]
os.environ["ALPHAVANTAGE_API_KEY"] = st.secrets["ALPHAVANTAGE_API_KEY"]

COHERE_API_KEY = st.secrets["COHERE_API_KEY"]
os.environ["COHERE_API_KEY"] = st.secrets["COHERE_API_KEY"]

QDRANT_HOST = st.secrets["QDRANT_HOST"]
QDRANT_API_KEY = st.secrets["QDRANT_API_KEY"]

OPEN_WEATHER_API_KEY = st.secrets["OPEN_WEATHER_API_KEY"]
os.environ["OPENWEATHERMAP_API_KEY"] = OPEN_WEATHER_API_KEY

POLYGON_API_KEY = st.secrets["POLYGON_API_KEY"]
os.environ["POLYGON_API_KEY"] = st.secrets["POLYGON_API_KEY"]


TAVILY_API_KEY = st.secrets["TAVILY_API_KEY"]
os.environ["TAVILY_API_KEY"] = st.secrets["TAVILY_API_KEY"]

Invoke ChatCohere

In [9]:
from langchain_cohere import ChatCohere
from langchain_core.messages import HumanMessage
chat = ChatCohere()
messages = [HumanMessage(content="knock knock")]
print(chat.invoke(messages))

content="Who's there?" additional_kwargs={'documents': None, 'citations': None, 'search_results': None, 'search_queries': None, 'is_search_required': None, 'generation_id': '8c7b69ad-1a4f-40aa-b365-910b679dc73f', 'token_count': {'input_tokens': 68, 'output_tokens': 4}} response_metadata={'documents': None, 'citations': None, 'search_results': None, 'search_queries': None, 'is_search_required': None, 'generation_id': '8c7b69ad-1a4f-40aa-b365-910b679dc73f', 'token_count': {'input_tokens': 68, 'output_tokens': 4}} id='run-3c2fb0b9-aba8-4378-b81a-03a069da25b2-0'


Invoke ChatCohere using ChatPromptTemplate

In [10]:
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_template("Tell me a joke about {topic}")
chain = prompt | chat
chain.invoke({"topic": "bears"})

AIMessage(content='Why don’t bears wear socks? \n\nThey have bear feet!', additional_kwargs={'documents': None, 'citations': None, 'search_results': None, 'search_queries': None, 'is_search_required': None, 'generation_id': '96aec7ab-7bb7-47c9-a38e-10d1ba5d99cf', 'token_count': {'input_tokens': 72, 'output_tokens': 15}}, response_metadata={'documents': None, 'citations': None, 'search_results': None, 'search_queries': None, 'is_search_required': None, 'generation_id': '96aec7ab-7bb7-47c9-a38e-10d1ba5d99cf', 'token_count': {'input_tokens': 72, 'output_tokens': 15}}, id='run-db8eb780-c8ee-4220-8bd5-75c9b3c37435-0')

Invoke Cohere

In [11]:
from langchain_cohere.llms import Cohere

llm = Cohere()
print(llm.invoke("Summarize the theory of Kahnemann and Tversky"))

 The theory of Kahnemann and Tversky is a collection of cognitive biases that affect how people make decisions and form beliefs. The theory consists of a list of heuristics (mental shortcuts) that are used by humans to make decisions, and the cognitive biases that result from these heuristics.

One of the most well-known heuristics is the availability heuristic, which is the tendency for people to make judgments based on how easy it is to think of examples for or against a particular issue. It is also known as the availability bias, which is a cognitive bias that causes people to overestimate the likelihood of events that are more easy to recall than events that are less easy to recall. For instance, individuals are more likely to believe that siblings resemble each other and that emigration rates are higher in the destination country when this country is next door, simply because these statements seem more available or easier to bring to mind. 

Let me know if you'd like to know more 

Tavily Internet Search Tool

In [12]:
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_cohere import ChatCohere, create_cohere_react_agent
from langchain_core.prompts import ChatPromptTemplate
from langchain.agents import AgentExecutor

llm = ChatCohere()

internet_search = TavilySearchResults(max_results=4)
internet_search.name = "internet_search"
internet_search.description = "Route a user query to the internet"

prompt = ChatPromptTemplate.from_template("{input}")

agent = create_cohere_react_agent(
    llm,
    [internet_search],
    prompt
)

agent_executor = AgentExecutor(agent=agent, tools=[internet_search], verbose=True)

agent_executor.invoke({
    "input": "When has Palantir had its IPO at the NYSE?",
})



> Entering new AgentExecutor chain...

I will search for 'Palantir IPO NYSE' and write an answer based on the information I find.
{'tool_name': 'internet_search', 'parameters': {'query': 'Palantir IPO NYSE'}}
[{'url': 'https://www.investopedia.com/palantir-ipo-what-you-need-to-know-5080082', 'content': "(If you want to hear us talk about it, check out this week's Express podcast)\n475.8 million Class A shares can be sold today (the remaining locked up, a rarity for direct listings) and the reference price is $7.25 a share, which translates to a valuation of nearly $16 billion. Financials\nControversy\nPalantir is seen as controversial because of the nature of some of the work its clients conduct, its inability to share much information on the projects and its unconventional corporate governance structure. What Does It Do\nNamed after the sinister seeing stones in the Lord of the Rings series, Palantir helps its clients analyze data to solve complicated problems or optimize operations

{'input': 'When has Palantir had its IPO at the NYSE?',
 'output': 'Palantir began trading on the New York Stock Exchange on Tuesday 30 September 2020.',
 'citations': [CohereCitation(start=57, end=64, text='Tuesday', documents=[{'url': 'https://finance.yahoo.com/news/palantir-technologies-ipo-investor-comprehensive-193224055.html', 'content': 'Palantir Stock IPO: A Direct Listing. Palantir stock begins trading on the NYSE on Tuesday, Sept. 30 under the ticker "PLTR". The company will pursue a direct listing, rather than a ...', 'id': 'doc_2'}], document_ids={'doc_2'}),
  CohereCitation(start=65, end=77, text='30 September', documents=[{'url': 'https://www.cnn.com/2020/09/30/tech/palantir-ipo/index.html', 'content': 'The company began trading on the New York Stock Exchange under the ticker symbol "PLTR" just after 1:30pm ET at $10 a share, up from the reference price listed by the NYSE of $7.25 a share ...', 'id': 'doc_1'}, {'url': 'https://finance.yahoo.com/news/palantir-technologies-

Wikipedia Tool

In [13]:
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

api_wrapper = WikipediaAPIWrapper(top_k_results=1)

class WikiInputs(BaseModel):
    """Inputs to the wikipedia tool."""

    query: str = Field(
        description="query to look up in Wikipedia"
    )
    
wikipedia_tool = WikipediaQueryRun(
    name="wiki-tool",
    description="look up things in wikipedia",
    args_schema=WikiInputs,
    api_wrapper=api_wrapper,
    return_direct=True,
)

prompt = ChatPromptTemplate.from_template("{input}")

agent = create_cohere_react_agent(
    llm,
    [wikipedia_tool],
    prompt
)

agent_executor = AgentExecutor(agent=agent, tools=[wikipedia_tool], verbose=True)

response = agent_executor.invoke({
                "input": "Explain the theory of Kahnemann and Tversky",
            })



> Entering new AgentExecutor chain...

I will search for 'Kahnemann and Tversky theory' and write an answer based on what I find.
{'tool_name': 'wiki-tool', 'parameters': {'query': 'Kahnemann and Tversky theory'}}
Page: Daniel Kahneman
Summary: Daniel Kahneman (; Hebrew: דניאל כהנמן; March 5, 1934 – March 27, 2024) was an Israeli-American cognitive scientist best-known for his work on the psychology of judgment and decision-making. He is also known for his work in behavioral economics, for which he was awarded the 2002 Nobel Memorial Prize in Economic Sciences together with Vernon L. Smith. Kahneman's published empirical findings challenge the assumption of human rationality prevailing in modern economic theory. Kahneman became known as the "grandfather of behavioral economics."
With Amos Tversky and others, Kahneman established a cognitive basis for common human errors that arise from heuristics and biases, and developed prospect theory. In 2011, Kahneman was named by Foreign Policy

Run tool without agent

In [14]:
wikipedia_tool.run("David Beckham")

"Page: David Beckham\nSummary: David Robert Joseph Beckham  ( BEK-əm; born 2 May 1975) is an English former professional footballer, the president and co-owner of Inter Miami CF and co-owner of Salford City. Primarily a right winger and known for his range of passing, crossing ability and free-kick taking, Beckham has been hailed as one of the greatest and most recognisable midfielders of his generation, as well as one of the best free-kick takers of all time. He won 19 major trophies in his career, and is the first English player to win league titles in four different countries: England, Spain, the United States and France.\nBeckham's professional club career began with Manchester United, where he made his first-team debut in 1992 at age 17. With United, he won the Premier League title six times, the FA Cup twice, the FA Charity Shield twice, the Intercontinental Cup and the UEFA Champions League in 1999. He then played four seasons with Real Madrid, winning the La Liga championship i

Openweathermap Tool

In [15]:
import os
import streamlit as st
from langchain_core.prompts import ChatPromptTemplate
from langchain.agents import AgentType, initialize_agent, load_tools
from langchain_cohere import ChatCohere, create_cohere_react_agent

llm = ChatCohere()

prompt = ChatPromptTemplate.from_template("{input}")


tools = load_tools(["openweathermap-api"], llm)
tools.append(wikipedia_tool)


agent = create_cohere_react_agent(
    tools=tools, llm=llm, prompt=prompt
)

agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

response = agent_executor.invoke({"input": "How is the weather in London?"})



> Entering new AgentExecutor chain...

I will use the Open Weather Map tool to find out the weather in London.
{'tool_name': 'open_weather_map', 'parameters': {'location': 'London'}}
In London, the current weather is as follows:
Detailed status: overcast clouds
Wind speed: 3.13 m/s, direction: 187°
Humidity: 84%
Temperature: 
  - Current: 16.68°C
  - High: 18.33°C
  - Low: 14.98°C
  - Feels like: 16.6°C
Rain: {}
Heat index: None
Cloud cover: 100%Relevant Documents: 0
Cited Documents: 0
Answer: The weather in London is currently overcast with a temperature of 16.68°C.
Grounded answer: The weather in London is currently <co: 0>overcast</co: 0> with a <co: 0>temperature of 16.68°C</co: 0>.

> Finished chain.


Access Output

In [16]:
response["output"]

'The weather in London is currently overcast with a temperature of 16.68°C.'

Multiple Tools

In [17]:
tools = load_tools(["openweathermap-api"], llm)
internet_search = TavilySearchResults(max_results=4)
internet_search.name = "internet_search"
internet_search.description = "Route a user query to the internet"
tools.append(internet_search)
tools.append(wikipedia_tool)

Multiple Tool Use
1. wiki-tool
2. open_weather_map
3. internet_search

In [21]:
prompt = ChatPromptTemplate.from_template("Use the right tool to answer the following question. If wikipedia_tool delivers no result use internet_search: {input}")
agent = create_cohere_react_agent(
    tools=tools, llm=llm, prompt=prompt
)

agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

print(agent_executor.invoke({"input": "Who was George Washington?"}))
print(agent_executor.invoke({"input": "What's the weather like in Mortugaba, Bahia, Brazil?"}))
print(agent_executor.invoke({"input": "Whats in the news today?"}))
print(agent_executor.invoke({"input": "How did the Microsoft stock perform last week?"}))



> Entering new AgentExecutor chain...

I will use the Wikipedia tool to answer this question.
{'tool_name': 'wiki-tool', 'parameters': {'query': 'Who was George Washington?'}}
Page: George Washington Carver
Summary: George Washington Carver (c. 1864 – January 5, 1943) was an American agricultural scientist and inventor who promoted alternative crops to cotton and methods to prevent soil depletion. He was one of the most prominent black scientists of the early 20th century.
While a professor at Tuskegee Institute, Carver developed techniques to improve types of soils depleted by repeated plantings of cotton. He wanted poor farmers to grow other crops, such as peanuts and sweet potatoes, as a source of their own food and to improve their quality of life. Under his leadership, the Experiment Station at Tuskegee published over forty practical bulletins for farmers, many written by him, which included recipes; many of the bulletins contained advice for poor farmers, including combating so

Alpha Vantage Tool

In [65]:
from langchain_community.utilities.alpha_vantage import AlphaVantageAPIWrapper
alpha_vantage = AlphaVantageAPIWrapper()
# alpha_vantage._get_exchange_rate("USD", "JPY")
# alpha_vantage._get_time_series_daily("MSFT")
# alpha_vantage._get_time_series_weekly("IBM")
alpha_vantage.search_symbols("American Superconductor")
# alpha_vantage._get_market_news_sentiment("Microsoft")
# alpha_vantage._get_top_gainers_losers()

{'bestMatches': [{'1. symbol': 'AMSC',
   '2. name': 'American Superconductor Corp',
   '3. type': 'Equity',
   '4. region': 'United States',
   '5. marketOpen': '09:30',
   '6. marketClose': '16:00',
   '7. timezone': 'UTC-04',
   '8. currency': 'USD',
   '9. matchScore': '0.9020'},
  {'1. symbol': 'AMS1.FRK',
   '2. name': 'American Superconductor Corporation',
   '3. type': 'Equity',
   '4. region': 'Frankfurt',
   '5. marketOpen': '08:00',
   '6. marketClose': '20:00',
   '7. timezone': 'UTC+02',
   '8. currency': 'EUR',
   '9. matchScore': '0.7931'}]}

Tool using Alpha Vantage

In [66]:
from langchain.agents import Tool

def search_symbols(query: str) -> str:
    """Searches a stock market symbol or a company."""
    alpha_vantage = AlphaVantageAPIWrapper()
    return alpha_vantage.search_symbols(query)



description = (
    "Useful for when you need to search for a company or a stock symbol. "
    "Useful if somebody asks 'How is the stock market symbol for...' "    
    "Useful if somebody asks 'For which company stands this stock symbol...' "
)

av_search_symbols = Tool(
    name="av_search_symbols",  # We'll just call it 'Data'
    func=search_symbols,
    description=description,
)

tools = load_tools(["openweathermap-api"], llm)
internet_search = TavilySearchResults(max_results=4)
internet_search.name = "internet_search"
internet_search.description = "Route a user query to the internet"
tools.append(internet_search)
tools.append(wikipedia_tool)
tools.append(av_search_symbols)



In [67]:
tools

[OpenWeatherMapQueryRun(api_wrapper=OpenWeatherMapAPIWrapper(owm=<pyowm.owm.OWM - API key=************************77bc3a62, subscription type=free, PyOWM version=(3, 3, 0)>, openweathermap_api_key=None)),
 TavilySearchResults(name='internet_search', description='Route a user query to the internet', max_results=4),
 WikipediaQueryRun(name='wiki-tool', description='look up things in wikipedia', args_schema=<class '__main__.WikiInputs'>, return_direct=True, api_wrapper=WikipediaAPIWrapper(wiki_client=<module 'wikipedia' from 'c:\\App\\Anaconda\\envs\\cohere\\Lib\\site-packages\\wikipedia\\__init__.py'>, top_k_results=1, lang='en', load_all_available_meta=False, doc_content_chars_max=4000)),
 Tool(name='av_search_symbols', description="Useful for when you need to search for a company or a stock symbol. Useful if somebody asks 'How is the stock market symbol for...' Useful if somebody asks 'For which company stands this stock symbol...' ", func=<function search_symbols at 0x0000017F00007420

In [68]:
prompt = ChatPromptTemplate.from_template("Use the right tool to answer the following question. If wikipedia_tool delivers no result use internet_search: {input}")
agent = create_cohere_react_agent(
    tools=tools, llm=llm, prompt=prompt
)

agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)


print(agent_executor.invoke({"input": "Whats in the news today?"}))
print(agent_executor.invoke({"input": "Tell me for which company stands the symbol MSFT"}))
print(agent_executor.invoke({"input": "What is the symbol for Cameco Corp?"}))



> Entering new AgentExecutor chain...

I will search for 'news 4 June 2024' and write an answer based on the results.
{'tool_name': 'internet_search', 'parameters': {'query': 'news 4 june 2024'}}
[{'url': 'https://www.state.gov/public-schedule-june-4-2024/', 'content': 'Assistant Secretary Stewart is on travel to Singapore and Qatar from May 29-June 5, 2024. BRIEFING SCHEDULE. 1:15 p.m. Department Press Briefing with Spokesperson Matthew Miller. (OPEN PRESS COVERAGE) The Department Press Briefing will be streamed live on the Department homepage and YouTube Channel. Office of the Spokesperson The Secretary of ...'}, {'url': 'https://www.usatoday.com/story/news/politics/elections/2024/05/30/2024-primary-election-preview/73890673007/', 'content': '1:24. The U.S. presidential primary season continues, with five states and the District of Columbia holding their elections on June 4. Voters in D.C., Iowa, Montana, New Jersey, New Mexico and ...'}, {'url': 'https://www.desmoinesregister.com/